In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import optuna

In [2]:
%run -i classes.ipynb

In [3]:
# Manual hyperparameters
epochs = 100
patience = 7
num_trials = 50 # For hyperparameter tuning

In [4]:
# Import and split the data
data = pd.read_csv('./data/train.csv')
X = data.drop('loan_status', axis=1)
y = data['loan_status']


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=420
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=69
)

preprocessor = LoanPreprocessor()
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)
X_test_processed = preprocessor.transform(X_test)

train_dataset = LoanDataset(X_train_processed, y_train.values)
val_dataset = LoanDataset(X_val_processed, y_val.values)
test_dataset = LoanDataset(X_test_processed, y_test.values)

input_size = X_train_processed.shape[1]

In [5]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Create an Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=num_trials)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)
print("Best accuracy: ", study.best_value)

# Create a DataFrame to store the results
results = []
for trial in study.trials:
    results.append({
        'Trial': trial.number,
        'Hidden Size 1': trial.params['hidden_size1'],
        'Hidden Size 2': trial.params['hidden_size2'],
        'Dropout Rate': trial.params['dropout_rate'],
        'Learning Rate': trial.params['learning_rate'],
        'Batch Size': trial.params['batch_size'],
        'Accuracy': trial.value
    })

results_df = pd.DataFrame(results)

# Display the results
print("\nHyperparameter Tuning Results:")
print(results_df)

# Optionally, save the results to a CSV file
results_df.to_csv('hyperparameter_tuning_results.csv', index=False)

[I 2024-12-20 20:56:55,112] A new study created in memory with name: no-name-6dcbaf60-7687-4f57-a868-05cb94886810


Epoch [1/100], Training Loss: 0.5285, Validation Loss: 0.3605
Best model weights saved.
Epoch [2/100], Training Loss: 0.3174, Validation Loss: 0.2839
Best model weights saved.
Epoch [3/100], Training Loss: 0.2707, Validation Loss: 0.2553
Best model weights saved.
Epoch [4/100], Training Loss: 0.2496, Validation Loss: 0.2408
Best model weights saved.
Epoch [5/100], Training Loss: 0.2377, Validation Loss: 0.2326
Best model weights saved.
Epoch [6/100], Training Loss: 0.2304, Validation Loss: 0.2276
Best model weights saved.
Epoch [7/100], Training Loss: 0.2252, Validation Loss: 0.2241
Best model weights saved.
Epoch [8/100], Training Loss: 0.2211, Validation Loss: 0.2210
Best model weights saved.
Epoch [9/100], Training Loss: 0.2176, Validation Loss: 0.2185
Best model weights saved.
Epoch [10/100], Training Loss: 0.2145, Validation Loss: 0.2160
Best model weights saved.
Epoch [11/100], Training Loss: 0.2118, Validation Loss: 0.2143
Best model weights saved.
Epoch [12/100], Training Loss:

[I 2024-12-20 20:58:57,190] Trial 0 finished with value: 0.9421393656928498 and parameters: {'hidden_size1': 107, 'hidden_size2': 19, 'dropout_rate': 0.44173865497201004, 'learning_rate': 3.33727134811877e-05, 'batch_size': 26}. Best is trial 0 with value: 0.9421393656928498.


Epoch [100/100], Training Loss: 0.1736, Validation Loss: 0.1877
Best model weights saved to best_model_trial_0.pth
Epoch [1/100], Training Loss: 0.2568, Validation Loss: 0.2243
Best model weights saved.
Epoch [2/100], Training Loss: 0.1981, Validation Loss: 0.2090
Best model weights saved.
Epoch [3/100], Training Loss: 0.1952, Validation Loss: 0.2062
Best model weights saved.
Epoch [4/100], Training Loss: 0.1917, Validation Loss: 0.1993
Best model weights saved.
Epoch [5/100], Training Loss: 0.1873, Validation Loss: 0.2088
Epoch [6/100], Training Loss: 0.1880, Validation Loss: 0.2100
Epoch [7/100], Training Loss: 0.1863, Validation Loss: 0.1979
Best model weights saved.
Epoch [8/100], Training Loss: 0.1843, Validation Loss: 0.2190
Epoch [9/100], Training Loss: 0.1841, Validation Loss: 0.1950
Best model weights saved.
Epoch [10/100], Training Loss: 0.1849, Validation Loss: 0.2007
Epoch [11/100], Training Loss: 0.1852, Validation Loss: 0.1919
Best model weights saved.
Epoch [12/100], Tra

[I 2024-12-20 20:59:13,688] Trial 1 finished with value: 0.9453222689553257 and parameters: {'hidden_size1': 128, 'hidden_size2': 61, 'dropout_rate': 0.31439226033671186, 'learning_rate': 0.03455724965645639, 'batch_size': 63}. Best is trial 1 with value: 0.9453222689553257.


...

Epoch [24/100], Training Loss: 0.1649, Validation Loss: 0.1872
Early stopping triggered.
Training stopped early.
Best model weights saved to best_model_trial_47.pth
Epoch [1/100], Training Loss: 0.3563, Validation Loss: 0.2543
Best model weights saved.
Epoch [2/100], Training Loss: 0.2385, Validation Loss: 0.2273
Best model weights saved.
Epoch [3/100], Training Loss: 0.2206, Validation Loss: 0.2178
Best model weights saved.
Epoch [4/100], Training Loss: 0.2112, Validation Loss: 0.2123
Best model weights saved.
Epoch [5/100], Training Loss: 0.2049, Validation Loss: 0.2075
Best model weights saved.
Epoch [6/100], Training Loss: 0.1997, Validation Loss: 0.2034
Best model weights saved.
Epoch [7/100], Training Loss: 0.1957, Validation Loss: 0.2000
Best model weights saved.
Epoch [8/100], Training Loss: 0.1928, Validation Loss: 0.1976
Best model weights saved.
Epoch [9/100], Training Loss: 0.1901, Validation Loss: 0.1962
Best model weights saved.
Epoch [10/100], Training Loss: 0.1884, Vali

[I 2024-12-20 21:16:17,787] Trial 48 finished with value: 0.9439581675571217 and parameters: {'hidden_size1': 37, 'hidden_size2': 57, 'dropout_rate': 0.29281621742619257, 'learning_rate': 0.00015830598278638233, 'batch_size': 34}. Best is trial 46 with value: 0.9464590201204957.


Epoch [40/100], Training Loss: 0.1727, Validation Loss: 0.1870
Early stopping triggered.
Training stopped early.
Best model weights saved to best_model_trial_48.pth
Epoch [1/100], Training Loss: 0.2325, Validation Loss: 0.2000
Best model weights saved.
Epoch [2/100], Training Loss: 0.1899, Validation Loss: 0.1964
Best model weights saved.
Epoch [3/100], Training Loss: 0.1847, Validation Loss: 0.1948
Best model weights saved.
Epoch [4/100], Training Loss: 0.1824, Validation Loss: 0.1902
Best model weights saved.
Epoch [5/100], Training Loss: 0.1801, Validation Loss: 0.1869
Best model weights saved.
Epoch [6/100], Training Loss: 0.1783, Validation Loss: 0.1896
Epoch [7/100], Training Loss: 0.1770, Validation Loss: 0.1878
Epoch [8/100], Training Loss: 0.1757, Validation Loss: 0.1913
Epoch [9/100], Training Loss: 0.1747, Validation Loss: 0.1886
Epoch [10/100], Training Loss: 0.1739, Validation Loss: 0.1862
Best model weights saved.
Epoch [11/100], Training Loss: 0.1725, Validation Loss: 0.

[I 2024-12-20 21:16:34,034] Trial 49 finished with value: 0.9438444924406048 and parameters: {'hidden_size1': 43, 'hidden_size2': 61, 'dropout_rate': 0.23910294834518048, 'learning_rate': 0.002255669305849787, 'batch_size': 40}. Best is trial 46 with value: 0.9464590201204957.


Epoch [20/100], Training Loss: 0.1650, Validation Loss: 0.1890
Early stopping triggered.
Training stopped early.
Best model weights saved to best_model_trial_49.pth
Best hyperparameters:  {'hidden_size1': 32, 'hidden_size2': 62, 'dropout_rate': 0.13635028177607467, 'learning_rate': 0.0017590017762048815, 'batch_size': 40}
Best accuracy:  0.9464590201204957

Hyperparameter Tuning Results:
    Trial  Hidden Size 1  Hidden Size 2  Dropout Rate  Learning Rate  \
0       0            107             19      0.441739       0.000033   
1       1            128             61      0.314392       0.034557   
2       2             77             53      0.338385       0.000062   
3       3             67             28      0.106339       0.023387   
4       4             55             45      0.312778       0.000136   
5       5             92             55      0.483788       0.000231   
6       6             95             36      0.468087       0.033269   
7       7             55         

In [7]:
# Load the best model weights
best_model_weights = torch.load(f"best_model_trial_{study.best_trial.number}.pth")

preprocessor = LoanPreprocessor()

# Load the best model
best_model = LoanNeuralNetwork(
    input_size=input_size, 
    hidden_size1=study.best_params['hidden_size1'], 
    hidden_size2=study.best_params['hidden_size2'], 
    dropout_rate=study.best_params['dropout_rate']
).to(device)

# Load the best model weights
best_model.load_state_dict(best_model_weights)

criterion = nn.BCELoss()

In [8]:
best_model.eval()
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
test_loss = validate_model(best_model, test_loader, criterion, device)

correct = 0
total = 0
with torch.no_grad():
    for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = best_model(features)
            predicted = (outputs.view(-1) > 0.5).float()  # Ensure outputs are flattened
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
print(f'Test Loss: {test_loss:.4f}')
print(f'Test accuracy: {test_accuracy:.4f}')

Test Loss: 0.1789
Test accuracy: 0.9475


In [9]:
# Train on all data with best hyperparameters
data = pd.read_csv('./data/train.csv')
X = data.drop('loan_status', axis=1)
y = data['loan_status']

preprocessor = LoanPreprocessor()
X_processed = preprocessor.fit_transform(X)

final_dataset = LoanDataset(X_processed, y.values)

# Train the final model
final_model = LoanNeuralNetwork(
    input_size=input_size, 
    hidden_size1=study.best_params['hidden_size1'], 
    hidden_size2=study.best_params['hidden_size2'], 
    dropout_rate=study.best_params['dropout_rate']
).to(device)

final_loader = DataLoader(final_dataset, batch_size=study.best_params['batch_size'], shuffle=True)
optimizer = optim.Adam(final_model.parameters(), lr=study.best_params['learning_rate'])

final_model.train()
early_stopping = EarlyStopping(patience=7, verbose=True)
best_loss = float('inf')
best_model_weights = None  # To store the best model weights

for epoch in range(epochs):
    total_loss = 0
    for features, labels in final_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = final_model(features)
        loss = criterion(outputs.view(-1), labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(final_loader)
    print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {avg_loss:.4f}')

    # Check early stopping
    early_stopping(avg_loss)
    if early_stopping.early_stop:
        print("Training stopped early.")
        break

    # Save the best model
    if avg_loss < best_loss:
        best_loss = avg_loss
        best_model_weights = final_model.state_dict()  # Save the model weights
        print("Best model weights saved.")

# Save the best model weights to a file
final_model_save_path = f"final_model.pth"
torch.save(best_model_weights, final_model_save_path)
print(f"Best model weights saved to {final_model_save_path}")

Epoch [1/100], Training Loss: 0.2295
Best model weights saved.
Epoch [2/100], Training Loss: 0.1983
Best model weights saved.
Epoch [3/100], Training Loss: 0.1924
Best model weights saved.
Epoch [4/100], Training Loss: 0.1887
Best model weights saved.
Epoch [5/100], Training Loss: 0.1855
Best model weights saved.
Epoch [6/100], Training Loss: 0.1850
Best model weights saved.
Epoch [7/100], Training Loss: 0.1846
Best model weights saved.
Epoch [8/100], Training Loss: 0.1831
Best model weights saved.
Epoch [9/100], Training Loss: 0.1822
Best model weights saved.
Epoch [10/100], Training Loss: 0.1808
Best model weights saved.
Epoch [11/100], Training Loss: 0.1813
Epoch [12/100], Training Loss: 0.1809
Epoch [13/100], Training Loss: 0.1800
Best model weights saved.
Epoch [14/100], Training Loss: 0.1795
Best model weights saved.
Epoch [15/100], Training Loss: 0.1790
Best model weights saved.
Epoch [16/100], Training Loss: 0.1787
Best model weights saved.
Epoch [17/100], Training Loss: 0.1777

In [10]:
# Load the test data for submission
raw_submission_data = pd.read_csv('./data/test.csv')
submission_data = preprocessor.fit_transform(raw_submission_data)

# Create the test dataset
submission_dataset = LoanDataset(submission_data, np.zeros(len(submission_data)))  # Placeholder for labels

# Create the test data loader
submission_loader = DataLoader(submission_dataset, batch_size=study.best_params['batch_size'], shuffle=False)


In [11]:
# Make predictions on the test data
predictions = []
final_model.eval()
with torch.no_grad():
    for features, _ in submission_loader:  # Ignore labels in test data
        features = features.to(device)
        outputs = final_model(features)
        predicted = (outputs.view(-1) > 0.5).float()  # Ensure outputs are flattened
        predictions.extend(predicted.cpu().numpy())

# Create a DataFrame to store the predictions
predictions_df = pd.DataFrame({
    'id': raw_submission_data['id'],  # Include the original 'id' column
    'loan_status': predictions
})

# Save the predictions to a CSV file
predictions_df.to_csv('test_predictions.csv', index=False)

print("Test predictions saved to test_predictions.csv")

Test predictions saved to test_predictions.csv
